In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torchvision
import imgaug as ia

from datasets import PandasDataset
from utils.transforms import Compose, dict_list_to_compose, RandomEdgeHomography, HOMOGRAPHY_RETURN
from utils.sampling import ConsecutiveSequences
from utils.io import generate_path, load_yaml

In [2]:
# # generate temporary dataset
# generate_path('tmp')

# # dict of videos
# # yml = load_yaml(path='configs/high_fps_with_camera_motion_videos_transforms.yml')
# yml = load_yaml(path='configs/high_fps_without_camera_motion_videos_transforms.yml')

# df = pd.DataFrame(columns=['filename'])

# i = 0
# for database in yml['databases']:
#     for idx, video in enumerate(database['videos']):
#         database['videos'][idx] = os.path.join(database['prefix'], video)

#     for idx, video in enumerate(database['videos']):
#         compose = dict_list_to_compose(database['transforms'][idx]) # generate compose transform from dict
#         consecutive_sequences = ConsecutiveSequences(paths=[video], max_seq=1, seq_stride=1, seq_len=1, transforms=[compose]) # generate iterator
        
#         for cs in consecutive_sequences:
#             filename = 'tmp/img_{}.png'.format(i)
#             cv2.imwrite(filename, cs[0])
#             df = df.append({'filename': filename}, ignore_index=True)
#             i += 1
# df.to_pickle('tmp/df.pkl')


In [3]:
df = pd.read_pickle('tmp/df.pkl')

ds_tf = PandasDataset(df, transforms=torchvision.transforms.Compose([
    RandomEdgeHomography(rho=64, crp_shape=[400, 533], homography_return=HOMOGRAPHY_RETURN.DATASET)
]))

ds = PandasDataset(df)

dl_tf = DataLoader(ds_tf, num_workers=8, batch_size=16)
dl = DataLoader(ds, num_workers=8, batch_size=16)

%timeit for dic in dl_tf: dic['img_crp'] = dic['img_crp'].cuda()
%timeit for img in dl: img = img.cuda()



1.89 s ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
900 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
